In [3]:
#inititalize the notebook
import os
import textwrap
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

chat_deployment_name = "gpt-4"

print("*** init done! ***")


*** init done! ***


simple prompt

In [22]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

In Nederland, waaronder Heerlen, duurt een voltijd HBO-ICT opleiding doorgaans
vier jaar. Dit geldt voor de meeste hogescholen die deze opleiding aanbieden.
Tijdens de opleiding ontwikkel je vaardigheden in verschillende gebieden binnen
de ICT, zoals software development, cybersecurity, data science en IT-service
management.  Er zijn ook opties voor deeltijd of duale opleidingen, en deze
kunnen langer duren afhankelijk van hoeveel tijd je elke week kunt besteden aan
je studie en of je dit combineert met werk.  Sommige studenten kunnen de
opleiding sneller afronden als zij bijvoorbeeld al relevante werkervaring
hebben, vrijstellingen krijgen of een versneld programma volgen. Anderzijds kan
het soms langer duren als studenten ervoor kiezen om extra stages te lopen, een
extra studiejaar in het buitenland in te lassen, bij studievertraging door
persoonlijke omstandigheden of als ze naast de opleiding werken en daardoor
minder studiepunten per jaar behalen.  Het is altijd een goed idee om d

system prompt

In [23]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent die antwoord als een grappige kat"
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Miauwachtig goed dat je het vraagt! De HBO-ICT opleiding in Heerlen – of waar
dan ook in Nederland – kruipt doorgaans over een periode van vier jaar. Dat is
als je het pad volgt zonder sluipweggetjes of kattenluikjes, zoals versnelde
programma's of deeltijdonderwijs. Maar geen zorgen, jonge katachtige padvinder,
elk jaar is gevuld met technologische snufjes en codetaal waar zelfs een slimme
kat zoals ik z'n snorharen voor zou omdraaien! Dus pak je laserpointer, zet je
geeky bril op, en bereid je voor op enkele jaren van hacken en snorren – uh, ik
bedoel uiteraard studeren. Succes!


RESET THE SYSTEM PROMPT

In [24]:
firstQuestion = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

firstResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": firstQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(firstResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Een HBO-ICT opleiding in Nederland, inclusief die in Heerlen, duurt gewoonlijk
vier jaar. Dit betreft een voltijd bacheloropleiding. Er bestaan ook
mogelijkheden om deze opleiding in deeltijd of als duaal traject te volgen, wat
zou kunnen betekenen dat de opleiding langer duurt, afhankelijk van de
hoeveelheid tijd die je per week aan de opleiding kunt besteden.  Daarnaast
bieden sommige hogescholen een versneld traject aan voor studenten met
bijvoorbeeld een vooropleiding op mbo-niveau 4 of na het behalen van propedeuse
op een andere hbo-opleiding. Zo'n traject kan de duur van de opleiding
verminderen, vaak tot ongeveer drie jaar.  Echter, het actuele aanbod en de
specifieke invulling van de opleiding kan verschillen per onderwijsinstelling.
Zo biedt Zuyd Hogeschool in Heerlen bijvoorbeeld de HBO-ICT opleiding aan. Het
is aan te raden de specifieke details te controleren op de website van de
onderwijsinstelling of direct contact op te nemen voor de meest actuele
informatie.


vervolgvraag

In [25]:
secondQuestion = "Wat is het adres van de campus?"

secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Om je vraag te beantwoorden over het adres van "de campus," zou ik meer
specifieke informatie nodig hebben over welke campus je bedoelt. Er zijn
wereldwijd duizenden campussen van universiteiten, hogescholen, scholen en
andere onderwijsinstellingen.   Als je de naam van de onderwijsinstelling of de
stad waar de campus zich bevindt kunt vermelden, zou ik je kunnen helpen het
juiste adres te vinden.


Add more context

In [26]:
systemWithMemory = {
    "role": "system",
    "content": f"""
            use the previous question and answer as a context 
            ##
            previous question {firstQuestion}
            previous answer {firstResponse.choices[0].message.content}
            """
}


secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        },
        systemWithMemory
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Het adres van de Zuyd Hogeschool, waar de HBO-ICT opleiding wordt aangeboden,
is:  Zuyd Hogeschool Nieuw Eyckholt 300 6419 DJ Heerlen Nederland  Zorg er wel
voor dat je de huidige informatie verifieert via de officiële
communicatiekanalen van Zuyd Hogeschool, omdat universiteiten en hogescholen
meerdere campussen of gebouwen kunnen hebben waar verschillende opleidingen
worden gegeven.


short term memory - langchain

In [27]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

load_dotenv()


# LLM
llm = AzureChatOpenAI(
    deployment_name=chat_deployment_name,
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
) 


# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Je bent een behulpzame assistent"
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
print("---------")
print(firstQuestion)
print("---------")
response = conversation({"question": firstQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)


print("---------")
print(secondQuestion + " (nu hebben we het antwoord van de eerste vraag als context)")
print("---------")
response = conversation({"question": secondQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)




---------
Hoe lang duurt een HBO-ICT opleiding in Heerlen?
---------
De HBO-ICT opleiding in Nederland, inclusief in Heerlen, duurt over het algemeen
4 jaar. Dit is de standaardduur voor een voltijd bacheloropleiding aan een
hogeschool in Nederland. Sommige studenten kunnen de opleiding sneller afronden
als ze bijvoorbeeld vrijstellingen krijgen of een versneld traject volgen.
Daarnaast is het ook mogelijk dat studenten er langer over doen, bijvoorbeeld
door een studievertraging of als ze de opleiding in deeltijd volgen.  Zuyd
Hogeschool is een voorbeeld van een instelling in Heerlen waar je een HBO-ICT
opleiding kunt volgen. Het is altijd goed om de specifieke informatie over de
opleiding te controleren bij de instelling zelf, omdat er specifieke programma's
of trajecten aangeboden kunnen worden die afwijken van de standaardduur.
---------
Wat is het adres van de campus? (nu hebben we het antwoord van de eerste vraag als context)
---------
Zuyd Hogeschool heeft meerdere campussen, maa

Je kunt nu gewoon verder chatten, memory wordt steeds verder uitgebreid

Let op! Je prompt wordt steeds groter, dus je verbruikt meer tokens
Wellicht heb je niet alle historie van de chat nodig.

dit is ook een voorbeeld hoe je een specifieke instructie geeft. "antwoord alleen met de link naar Google Maps" is een hele specifieke instructie.

In [33]:
thirdQuestion = """
                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps
                """
print("---------")
print(thirdQuestion)
print("---------")
response = conversation({"question": thirdQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)



---------

                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps
                
---------
https://www.google.com/maps?q=Nieuw+Eyckholt+300,+6419+DJ+Heerlen,+Nederland


nog een voorbeeld van een hele specifieke instructie

In [54]:
JsonQuestion = """
                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                """
print("---------")
print(JsonQuestion)
print("---------")
response = conversation({"question": JsonQuestion})

# we need to catch errors and handle them, because model output could be invalid json
# most of the time, this will just work
try:
    address = json.loads(response["text"])

    print(address["street"])    
    print(address["postal_code"])
    print(address["city"])
    print(address["country"])
    
except Exception as e:
    print("Error: " + str(e))

---------

                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                
---------
Nieuw Eyckholt 300
6419 DJ
Heerlen
Nederland


let's get creative!

In [2]:
synthDataQuestion = """
        I'm building a quiz about computer games?
        Can you provide me with some data?
        I need 5 questions on recent games with 4 answers each, only one of them is correct

        ## sample json
        {
            "question": "question?",
            "answers": [
                "answer1",
                "answer2",
                "answer3",
                "answer4"
            ],
            "correct_answer": 2
        }
"""

synthResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": synthDataQuestion
        },
        {
            "role": "system",
            "content" : """
                        You are a helpfull assistant that only outputs 
                        requested data in JSON format
                        output a plain json as provided in the example by the user
                        """
        }
    ]
)

print(synthResponse.choices[0].message.content)



```json
[
    {
        "question": "What is the main character's name in the game 'Cyberpunk 2077'?",
        "answers": [
            "Johnny Silverhand",
            "V",
            "Geralt of Rivia",
            "Aloy"
        ],
        "correct_answer": 1
    },
    {
        "question": "Which game, released in 2020, is a multiplayer social deduction game developed by InnerSloth?",
        "answers": [
            "Fall Guys: Ultimate Knockout",
            "Valorant",
            "Among Us",
            "Genshin Impact"
        ],
        "correct_answer": 2
    },
    {
        "question": "In the 2021 game 'Resident Evil Village', who is the primary antagonist?",
        "answers": [
            "Mother Miranda",
            "Lady Dimitrescu",
            "Chris Redfield",
            "Ethan Winters"
        ],
        "correct_answer": 0
    },
    {
        "question": "Which of these games features a massive open-world island known as Tsushima, under Mongol invasion?",
  

Een hele specifieke vraag

In [14]:
mySpecificQuestion = "Wat is er speciaal aan de vrijdag voor de opleiding HBO-ICT in Heerlen?"

mySpecificResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": mySpecificQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)


lines = textwrap.wrap(mySpecificResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)



Ik kan geen specifieke details geven over wat er speciaal zou kunnen zijn aan de
vrijdag voor de opleiding HBO-ICT in Heerlen zonder actuele informatie. Echter,
onderwijsinstellingen organiseren vaak speciale evenementen, workshops,
gastcolleges of andere activiteiten op bepaalde dagen van de week die relevant
kunnen zijn voor studenten.  Als je geïnteresseerd bent in wat er specifiek op
vrijdagen gebeurt voor de HBO-ICT opleiding in Heerlen, zou ik aanraden om de
website van de betreffende onderwijsinstelling te bezoeken of direct contact op
te nemen met de faculteit of het secretariaat van de opleiding voor de meest
actuele informatie. Zij kunnen je voorzien van een overzicht van geplande
evenementen of bijzondere kenmerken van de opleiding op die dag.


Wellicht kun je wat meer context geven in de prompt.

We gaan nu data toevoegen, hele specifieke data, namelijk een aantal pagina's
van de HSZuyd Website.

In [12]:
#lees de pagina's om deze later toe te voegen in de prompt
page1 = open("../docs/page-0.md", "r").read()
page2 = open("../docs/page-1.md", "r").read()
page3 = open("../docs/page-2.md", "r").read()


myDataQuestion = "Wat is er speciaal aan de vrijdag voor de opleiding HBO-ICT in Heerlen?"

myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {page1}

                ## data
                {page2}

                ## data
                {page3}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)

Op de vrijdag tijdens de opleiding HBO-ICT in Heerlen is er een evaluatiedag. Op
deze dag wordt gekeken of de studenten alles kennen en of alle opdrachten af
zijn. Ook wordt er aandacht besteed aan de samenwerking binnen de groep. Deze
dag draait ook om 'professional skills'. Studenten laten zien wat ze hebben
gedaan en soms komt er een ervaren ICT'er vertellen over zijn werk. Het is dus
een dag waarop reflectie en professionele ontwikkeling centraal staan.
----
Hier wat info over de tokens die je hebt gebruikt:
CompletionUsage(completion_tokens=123, prompt_tokens=11853, total_tokens=11976)


Okay....dit lijkt werken, maar we verbruiken heel veel tokens...tijd voor een volgende stap.

***Embeddings en vectoren